In [ ]:
!pip install transformers[torch] accelerate datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 13.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.w

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


Using device: cuda


In [ ]:
from datasets import load_dataset

dataset = load_dataset('wmt18', 'zh-en', split='train[:1%]')
print(f"数据集大小: {len(dataset)}")

# 打印前几个示例以确认数据加载成功
print(f"数据集示例: {dataset[:3]}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/25160346 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3981 [00:00<?, ? examples/s]

数据集大小: 251603
数据集示例: {'translation': [{'en': '1929 or 1989?', 'zh': '1929年还是1989年?'}, {'en': 'PARIS – As the economic crisis deepens and widens, the world has been searching for historical analogies to help us understand what has been happening.', 'zh': '巴黎-随着经济危机不断加深和蔓延，整个世界一直在寻找历史上的类似事件希望有助于我们了解目前正在发生的情况。'}, {'en': 'At the start of the crisis, many people likened it to 1982 or 1973, which was reassuring, because both dates refer to classical cyclical downturns.', 'zh': '一开始，很多人把这次危机比作1982年或1973年所发生的情况，这样得类比是令人宽心的，因为这两段时期意味着典型的周期性衰退。'}]}


max_length 这边不宜设定太高这样后面训练的时候容易内存爆掉，参数用于指定切分后的文本序列的最大长度。如果输入文本的长度超过了max_length，则会进行截断（truncation）以确保序列的长度不超过max_length。如果输入文本的长度不足max_length，将会进行填充（padding）以使序列长度一致。

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')

def preprocess_function(examples):
    inputs = [ex['zh'] for ex in examples['translation']]
    targets = [ex['en'] for ex in examples['translation']]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")
    labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

encoded_dataset = dataset.map(preprocess_function, batched=True)

# 打印预处理后的前几个示例
print(f"预处理后的数据集示例: {encoded_dataset[:3]}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

Map:   0%|          | 0/251603 [00:00<?, ? examples/s]

预处理后的数据集示例: {'translation': [{'en': '1929 or 1989?', 'zh': '1929年还是1989年?'}, {'en': 'PARIS – As the economic crisis deepens and widens, the world has been searching for historical analogies to help us understand what has been happening.', 'zh': '巴黎-随着经济危机不断加深和蔓延，整个世界一直在寻找历史上的类似事件希望有助于我们了解目前正在发生的情况。'}, {'en': 'At the start of the crisis, many people likened it to 1982 or 1973, which was reassuring, because both dates refer to classical cyclical downturns.', 'zh': '一开始，很多人把这次危机比作1982年或1973年所发生的情况，这样得类比是令人宽心的，因为这两段时期意味着典型的周期性衰退。'}], 'input_ids': [[101, 9792, 2399, 6820, 3221, 8528, 2399, 136, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [101, 2349, 7944, 118, 7390, 4708, 5307,

In [ ]:
from transformers import T5ForConditionalGeneration, TrainingArguments, Trainer

model = T5ForConditionalGeneration.from_pretrained('t5-small', force_download=True).to(device)

training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=128,  # 增加批处理大小A100 128最多会爆
    per_device_eval_batch_size=128,   # 增加批处理大小A100 128最多会爆
    num_train_epochs=10,  # 设置为1个epoch进行快速测试
    weight_decay=0.01,
    fp16=True,  # 启用混合精度训练
    gradient_accumulation_steps=2,  # 累积2步梯度
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset,
    eval_dataset=encoded_dataset,
)
print("开始训练模型...")
trainer.train()
print("训练完成。")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

开始训练模型...


Epoch,Training Loss,Validation Loss
1,3.949000,2.854347
2,2.881700,2.660603
3,2.717800,2.474664
4,2.571600,2.331986
5,2.459800,2.236256
6,2.376100,2.170124
7,2.322100,2.125072
8,2.284100,2.096364
9,2.259600,2.079910
10,2.247500,2.074291


训练完成。


In [ ]:
import matplotlib.pyplot as plt

def translate_texts(texts):
    # 确保模型在正确的设备上
    model.to(device)

    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True).to(device)
    # 删除不必要的参数
    inputs = {k: v for k, v in inputs.items() if k in ['input_ids', 'attention_mask']}

    # 检查输入张量
    print(f"输入张量: {inputs}")

    # 调整生成参数
    outputs = model.generate(**inputs, max_length=128, num_beams=10, early_stopping=True)

    # 检查生成的输出张量
    print(f"生成的输出张量: {outputs}")

    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

examples = ["巴黎-随着经济危机不断加深和蔓延，整个世界一直在寻找历史上的类似事件希望有助于我们了解目前正在发生的情况。"]
translations = translate_texts(examples)

# 检查翻译结果
for i, (src, tgt) in enumerate(zip(examples, translations)):
    print(f"示例 {i+1}")
    print(f"中文: {src}")
    print(f"英文: {tgt}")
    print("-" * 20)


输入张量: {'input_ids': tensor([[ 101, 2349, 7944,  118, 7390, 4708, 5307, 3845, 1314, 3322,  679, 3171,
         1217, 3918, 1469, 5913, 2454, 8024, 3146,  702,  686, 4518,  671, 4684,
         1762, 2192, 2823, 1325, 1380,  677, 4638, 5102,  849,  752,  816, 2361,
         3307, 3300, 1221,  754, 2769,  812,  749, 6237, 4680, 1184, 3633, 1762,
         1355, 4495, 4638, 2658, 1105,  511,  102]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
生成的输出张量: tensor([[   0,  101,  100,  117, 8174,  100,  100,  100,  100,  100,  100,  100,
          100,  100,  100,  100,  100,  100,  100,  100,  100,  100,  100,  100,
          100,  100,  161,  100,  100,  100,  100,  100,  100,  100,  100,  100,
          100,  100,  100,  100,  100,  100,  100,  100,  100,  100,  100,  100,
          100,  100,  

In [ ]:
examples = ["一开始，很多人把这次危机比作1982年或1973年所发生的情况，这样得类比是令人宽心的，因为这两段时期意味着典型的周期性衰退。"]
translations = translate_texts(examples)

# 检查翻译结果
for i, (src, tgt) in enumerate(zip(examples, translations)):
    print(f"示例 {i+1}")
    print(f"中文: {src}")
    print(f"英文: {tgt}")
    print("-" * 20)

输入张量: {'input_ids': tensor([[ 101,  671, 2458, 1993, 8024, 2523, 1914,  782, 2828, 6821, 3613, 1314,
         3322, 3683,  868, 8687, 2399, 2772, 9062, 2399, 2792, 1355, 4495, 4638,
         2658, 1105, 8024, 6821, 3416, 2533, 5102, 3683, 3221,  808,  782, 2160,
         2552, 4638, 8024, 1728,  711, 6821,  697, 3667, 3198, 3309, 2692, 1456,
         4708, 1073, 1798, 4638, 1453, 3309, 2595, 6139, 6842,  511,  102]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
生成的输出张量: tensor([[   0,  101,  100,  117, 8174,  100,  100,  100,  100,  100,  100,  100,
          100,  100,  100,  100,  100,  100,  100,  100,  100,  100,  100,  100,
          161,  100,  100,  100,  100,  100,  100,  100,  100,  100,  100,  100,
          100,  100,  100,  100,  100,  100,  100,  100,  10

In [ ]:

examples = ["今天是个好天气","令人宽心的","1929年还是1989年?"]
translations = translate_texts(examples)

# 检查翻译结果
for i, (src, tgt) in enumerate(zip(examples, translations)):
    print(f"示例 {i+1}")
    print(f"中文: {src}")
    print(f"英文: {tgt}")
    print("-" * 20)


输入张量: {'input_ids': tensor([[ 101,  791, 1921, 3221,  702, 1962, 1921, 3698,  102],
        [ 101,  808,  782, 2160, 2552, 4638,  102,    0,    0],
        [ 101, 9792, 2399, 6820, 3221, 8528, 2399,  136,  102]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 0, 0],
        [1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}
生成的输出张量: tensor([[  0, 101, 100, 100, 102,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0